In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade keras

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import smtplib

model=load_model('trained_facemask.h5')
face_det_classifier=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
text_dict={0:'Mask ON',1:'Mask OFF'}
rect_color_dict={0:(0,255,0),1:(0,0,255)}

# SUBJECT="Subject"
# TEXT="ACCESS DENIED"

if not cap.isOpened():
    raise IOError("Cannot open webcam")

while(True):
    ret, img = cap.read()
    grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_det_classifier.detectMultiScale(grayscale_img,1.3,5)
    for(x,y,w,h) in faces:
        face_img=grayscale_img[y:y+w,x:x+w]
        #resized_img=cv2.resize(face_img,(112,112),fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        try:
            resized_img=cv2.resize(face_img,(112,112),fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        except Exception as e:
            print(str(e))
        normalized_img=resized_img/255.0
        reshaped_img=np.reshape(normalized_img,(1,112,112,1))
        result=model.predict(reshaped_img)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
#         if (label==1):
#             message='Subject: {}\n\n{}'.format(SUBJECT,TEXT)
#             mail=smtplib.SMTP('smtp.gmail.com',587)
#             mail.ehlo()
#             mail.starttls()
#             mail.login('lizzy.darcy.harry.potter@gmail.com','prideandprejudice')
#             mail.sendmail('tahseena.mahmud@gmail.com','lizzy.darcy.harry.potter@gmail.com',message)
#         else:
#             pass
#             break
    cv2.imshow('CCTV',img)
    key=cv2.waitKey(1)
    if(key==27):
        break

cap.release()
cv2.destroyAllWindows()